# Import Libraries

In [ ]:
from arcgis.gis import GIS
import getpass

from datetime import datetime, timedelta


# Connect to AGOL

In [ ]:
# Input username, password and group owner username
username = input("Enter your username: ")
password = getpass.getpass("Enter your password: ")


# Connect to the GIS
gis = GIS("https://www.arcgis.com", username, password)


In [ ]:

# Get current time in milliseconds since epoch
current_time_ms = int(time.time() * 1000)

# Calculate 5 years in milliseconds
five_years_in_ms = 5 * 365 * 24 * 60 * 60 * 1000  # Not accounting for leap years for simplicity

# Query all users (or as many as possible)
all_users = gis.users.search(max_users=10000)  # Adjust max_users as needed

# Filter users based on lastLogin
users_not_logged_in_5_years = [user for user in all_users if user.lastLogin < (current_time_ms - five_years_in_ms)]

# Check the number of users found
print(f"Found {len(users_not_logged_in_5_years)} users who haven't logged in in the last 5 years.")

In [ ]:
# Assuming users_not_logged_in_5_years is already populated with user objects
list_of_user_emails = [user.email for user in users_not_logged_in_5_years if user.email is not None]

# Print the list of user emails
print(list_of_user_emails)

# User items & Groups

In [ ]:
# Initialize dictionaries outside the loop
all_member_group_names = {}
all_owned_group_names = {}
all_item_titles = {}

for user_email in list_of_user_emails:
    # Get the User object for the current user by email
    current_user = gis.users.search(query=f"email:{user_email}")[0]  # Assuming the first result is the desired user
    username = current_user.username  # Get the username of the current user

    # Initialize lists in dictionaries if the username is not already a key
    if username not in all_member_group_names:
        all_member_group_names[username] = []
    if username not in all_owned_group_names:
        all_owned_group_names[username] = []
    if username not in all_item_titles:
        all_item_titles[username] = []

    # Get all groups where the current user is a member
    member_groups = current_user.groups
    # Get all groups owned by the current user
    owned_groups = gis.groups.search(query=f"owner:{username}")
    # Get all items owned by the current user
    items = gis.content.search(query=f"owner:{username}", max_items=9999)

    # Append group names and item titles to the corresponding lists in the dictionaries
    all_member_group_names[username].extend([group.title for group in member_groups])
    all_owned_group_names[username].extend([group.title for group in owned_groups])
    all_item_titles[username].extend([item.title for item in items])

# Reassign Items and groups

In [ ]:
# Input new owner username
new_owner_username = "retiredcontent_esriaiddev"

# Initialize a list to store items that fail to reassign
failed_items = []

# Iterate through the usernames in the dictionary
for username in all_owned_group_names.keys():
    # Get the User object if not already
    current_user = gis.users.get(username)

    # Get the new owner User object
    new_owner_user = gis.users.get(new_owner_username)

    # Prepare the folder for the new owner
    folder_name = f"{username}_items"
    # Check if the folder exists, if not, create it
    existing_folders = new_owner_user.folders
    folder_names = [folder['title'] for folder in existing_folders]
    if folder_name not in folder_names:
        gis.content.create_folder(folder=folder_name, owner=new_owner_username)

    # Reassign groups to the new owner
    for group_title in all_owned_group_names[username]:
        # Check if a group with the same name already exists under the new owner
        existing_group = gis.groups.search(query=f"title:{group_title} AND owner:{new_owner_username}")
        if existing_group:
            print(f"Skipping reassignment for group '{group_title}' as it already exists under the new owner.")
            continue  # Skip this group

        # Search for the group by title to reassign
        group_search_result = gis.groups.search(query=f"title:{group_title} AND owner:{username}")
        if group_search_result:
            group = group_search_result[0]
            try:
                # Check and disable delete protection if enabled
                if group.protected:
                    group.protected = False
                    print(f"Delete protection disabled for group '{group_title}'.")

                group.reassign_to(new_owner_username)
            except Exception as e:
                print(f"Error reassigning group '{group_title}': {e}")

    for item_title in all_item_titles[username]:
        try:
            item_search_result = gis.content.search(query=f"title:{item_title} AND owner:{username}", max_items=1)
            if item_search_result:
                item = item_search_result[0]
                # Attempt to reassign related views first
                views_search_result = gis.content.search(query=f"type:'Feature Service' AND owner:{username} AND isView:true AND parentItemId:{item.id}")
                for view in views_search_result:
                    try:
                        view.reassign_to(new_owner_username)
                        view.move(folder=folder_name)
                        print(f"Successfully reassigned view {view.title} to {new_owner_username}")
                    except Exception as view_error:
                        print(f"Failed to reassign view {view.title} due to: {view_error}")
                # Now attempt to reassign the main item
                item.reassign_to(new_owner_username)
                item.move(folder=folder_name)
                print(f"Successfully reassigned {item.title} to {new_owner_username} in folder {folder_name}")
        except Exception as item_error:
            print(f"Failed to reassign {item_title} due to: {item_error}")
            # Add the failed item to the list
            failed_items.append((item_title, username))

# Optionally, remove the old user from their groups if needed
for group_name in all_member_group_names[username]:
    group_search_result = gis.groups.search(query=f"title:{group_name} AND owner:{username}")
    if group_search_result:
        group = group_search_result[0]
        group.remove_users([username])


In [ ]:
for username in all_owned_group_names.keys():
    user_groups = gis.groups.search(query=f"owner:{username}")
    for group in user_groups:
        try:
            group_title = group.title  # Store the group's title before deletion
            if group.protected:
                group.protected = False
                print(f"Delete protection disabled for group '{group_title}'.")
            group.delete()
            print(f"Group '{group_title}' deleted successfully.")  # Use the stored title here
        except Exception as e:
            if "Group does not exist or is inaccessible." in str(e) or "Error Code: 400" in str(e):
                print(f"Group '{group.title}' does not exist or is inaccessible. Skipping.")
            else:
                print(f"Failed to delete group '{group.title}'. Error: {e}")

Run the below code only if you want to delete the unassigned items.

In [ ]:

# Attempt to reassign failed items again
for item_title, username in failed_items:
    try:
        item_search_result = gis.content.search(query=f"title:{item_title} AND owner:{username}", max_items=1)
        if item_search_result:
            item = item_search_result[0]
            item.reassign_to(new_owner_username)
            item.move(folder=folder_name)
            print(f"Successfully reassigned {item_title} to {new_owner_username} in folder {folder_name} on second attempt")
    except Exception as e:
        print(f"Failed to reassign {item_title} on second attempt due to: {e}. Checking for delete protection.")
        # Check for delete protection
        if item.protected:
            # Disable delete protection
            item.protected = False
            print(f"Delete protection disabled for {item_title}.")
        # Attempt to delete the item
        try:
            item.delete()
            print(f"Item {item_title} deleted successfully.")
        except Exception as delete_error:
            print(f"Failed to delete {item_title} due to: {delete_error}.")
    

In [ ]:

# Extract usernames from the dictionary keys
usernames_to_delete = list(all_item_titles.keys())

# Iterate through the list of usernames to delete
for username in usernames_to_delete:
    try:
        # Search for the user
        user = gis.users.get(username)
        if user:
            # Optional: Transfer ownership of items before deleting the user
            # This step is skipped in this example for brevity

            # Delete the user
            user.delete()
            print(f"User {username} deleted successfully.")
        else:
            print(f"User {username} not found.")
    except Exception as e:
        print(f"Failed to delete user {username}. Error: {e}")